# 01 — Build State-Year Panel

Loads **pre-fetched** SAIPE state-year data (parquet) and BLS LAUS unemployment (txt), cleans them, merges into a single panel, and saves to `data/processed/`.

In [1]:
import sys, platform
import pandas as pd
import numpy as np

print("python:", sys.version.split()[0])
print("platform:", platform.platform())
print("pandas:", pd.__version__)
print("numpy:", np.__version__)

python: 3.12.9
platform: Windows-11-10.0.26200-SP0
pandas: 2.3.3
numpy: 2.3.5


In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

# =====================================================
# Repo + paths (works even if kernel starts outside repo)
# =====================================================

START = Path.cwd().resolve()

def find_repo(start: Path) -> Path:
    # Find repo root by locating data/raw containing required inputs.
    roots = [start]
    if start.drive:
        roots.append(Path(start.drive + "\\") / "projects")

    required_raw_files = [
        "saipe_state_year.parquet",
        "laus_allstates_u.txt",
        "la.area.txt",
        "la.series.txt",
    ]

    seen = set()
    for root in roots:
        root = root.resolve()
        if root in seen or not root.exists():
            continue
        seen.add(root)

        for raw_dir in root.rglob("data/raw"):
            if all((raw_dir / f).exists() for f in required_raw_files):
                return raw_dir.parent.parent  # .../data/raw -> repo root

    raise RuntimeError(
        f"Could not find repo root from start={start}. "
        f"Expected data/raw containing: {required_raw_files}"
    )

REPO = find_repo(START)
RAW = REPO / "data" / "raw"
PROCESSED = REPO / "data" / "processed"
PROCESSED.mkdir(parents=True, exist_ok=True)

SAIPE_PATH = RAW / "saipe_state_year.parquet"
LAUS_DATA_PATH = RAW / "laus_allstates_u.txt"
LAUS_AREA_PATH = RAW / "la.area.txt"
LAUS_SERIES_PATH = RAW / "la.series.txt"

print("CWD:", START)
print("Repo:", REPO)
print("Raw files:", [p.name for p in sorted(RAW.glob('*'))])
print("Processed dir:", PROCESSED)

for p in [SAIPE_PATH, LAUS_DATA_PATH, LAUS_AREA_PATH, LAUS_SERIES_PATH]:
    assert p.exists(), f"Missing required input: {p}"

print("✅ Found all required raw inputs")

CWD: C:\projects\python-policy-project\notebooks
Repo: C:\projects\python-policy-project
Raw files: ['la.area.txt', 'la.series.txt', 'laus_allstates_u.txt', 'saipe_state_year.parquet']
Processed dir: C:\projects\python-policy-project\data\processed
✅ Found all required raw inputs


In [3]:
# =====================================================
# SAIPE: load pre-fetched, cleaned state-year parquet
# =====================================================

saipe = pd.read_parquet(SAIPE_PATH)

expected = {"state", "state_name", "state_fips", "year", "poverty_rate", "median_income"}
missing = expected - set(saipe.columns)
assert not missing, f"SAIPE is missing columns: {missing}"

# Types
saipe["year"] = pd.to_numeric(saipe["year"], errors="raise").astype(int)
saipe["poverty_rate"] = pd.to_numeric(saipe["poverty_rate"], errors="coerce")
saipe["median_income"] = pd.to_numeric(saipe["median_income"], errors="coerce")

# Clean final
saipe_clean = (
    saipe[["state", "state_name", "state_fips", "year", "poverty_rate", "median_income"]]
    .dropna(subset=["state", "state_name", "year", "poverty_rate", "median_income"])
    .drop_duplicates(subset=["state", "year"])
    .sort_values(["state", "year"])
    .reset_index(drop=True)
)

print("SAIPE shape:", saipe_clean.shape)
print("SAIPE columns:", saipe_clean.columns.tolist())
print("SAIPE duplicate state-year rows:", int(saipe_clean.duplicated(["state", "year"]).sum()))

states_per_year = saipe_clean.groupby("year")["state"].nunique()
print("States per year (tail):")
print(states_per_year.tail())

# NOTE: 51 = 50 states + DC (Census SAIPE includes DC)
assert states_per_year.min() == 51, f"Expected 51 states+DC each year, got min={states_per_year.min()}"

display(saipe_clean.head())

SAIPE shape: (1581, 6)
SAIPE columns: ['state', 'state_name', 'state_fips', 'year', 'poverty_rate', 'median_income']
SAIPE duplicate state-year rows: 0
States per year (tail):
year
2019    51
2020    51
2021    51
2022    51
2023    51
Name: state, dtype: int64


,state,state_name,state_fips,year,poverty_rate,median_income
0,AK,Alaska,02,1989,10.6,33885
1,AK,Alaska,02,1993,11.2,39431
2,AK,Alaska,02,1995,10.1,42255
3,AK,Alaska,02,1996,10.6,44797
4,AK,Alaska,02,1997,11.2,43657


In [4]:
# =====================================================
# LAUS: State-level unemployment rate (annual averages)
# =====================================================

def load_laus_tsv(path):
    # Load BLS tab-delimited files and strip whitespace everywhere.
    df = pd.read_csv(path, sep="\t", dtype=str)
    df.columns = df.columns.str.strip()
    for c in df.columns:
        df[c] = df[c].astype(str).str.strip()
    return df

laus      = load_laus_tsv(LAUS_DATA_PATH)
la_area   = load_laus_tsv(LAUS_AREA_PATH)
la_series = load_laus_tsv(LAUS_SERIES_PATH)

print("LAUS rows:", laus.shape, "| cols:", laus.columns.tolist())
print("AREA rows:", la_area.shape, "| cols:", la_area.columns.tolist())
print("SERIES rows:", la_series.shape, "| cols:", la_series.columns.tolist())

# 1) State lookup (area_code → state_name)
la_states = (
    la_area
    .loc[la_area["area_type_code"] == "A", ["area_code", "area_text"]]
    .rename(columns={"area_text": "state_name"})
)

# 2) Unemployment RATE series (keep area_code)
urate_series = (
    la_series
    .loc[la_series["measure_code"] == "03", ["series_id", "area_code"]]
)

# 3) Merge LAUS data with series metadata (adds area_code), keep annual avg (M13)
laus_merged = (
    laus
    .merge(urate_series, on="series_id", how="inner")
    .loc[lambda d: d["period"] == "M13"]
)

# 4) Attach state name using area_code
laus_merged = laus_merged.merge(la_states, on="area_code", how="inner")

# 5) Final clean
laus_merged["year"] = pd.to_numeric(laus_merged["year"], errors="coerce").astype("Int64")
laus_merged["unemployment_rate"] = pd.to_numeric(laus_merged["value"], errors="coerce")

laus_clean = (
    laus_merged[["state_name", "year", "unemployment_rate"]]
    .dropna()
    .drop_duplicates(subset=["state_name", "year"])
    .sort_values(["state_name", "year"])
    .reset_index(drop=True)
)

# Drop Puerto Rico (keep 50 + DC)
laus_clean = laus_clean[laus_clean["state_name"] != "Puerto Rico"].copy()

# Map state_name -> USPS using SAIPE mapping
name_to_usps = dict(zip(saipe_clean["state_name"], saipe_clean["state"]))
laus_clean["state"] = laus_clean["state_name"].map(name_to_usps)

bad = laus_clean.loc[laus_clean["state"].isna(), "state_name"].value_counts()
if len(bad) > 0:
    raise ValueError(f"Unmapped state names coming from LAUS:\n{bad}")

laus_clean["year"] = laus_clean["year"].astype(int)
laus_clean = laus_clean[["state", "year", "unemployment_rate"]].copy()

print("LAUS clean shape:", laus_clean.shape)
print("LAUS states covered:", laus_clean["state"].nunique())
print("Duplicate state-year rows:", int(laus_clean.duplicated(["state","year"]).sum()))
print("States per year (tail):")
print(laus_clean.groupby("year")["state"].nunique().tail())

display(laus_clean.sample(1, random_state=0))

LAUS rows: (233206, 5) | cols: ['series_id', 'year', 'period', 'value', 'footnote_codes']
AREA rows: (8325, 6) | cols: ['area_type_code', 'area_code', 'area_text', 'display_level', 'selectable', 'sort_sequence']
SERIES rows: (33881, 12) | cols: ['series_id', 'area_type_code', 'area_code', 'measure_code', 'seasonal', 'srd_code', 'series_title', 'footnote_codes', 'begin_year', 'begin_period', 'end_year', 'end_period']
LAUS clean shape: (2499, 3)
LAUS states covered: 51
Duplicate state-year rows: 0
States per year (tail):
year
2020    51
2021    51
2022    51
2023    51
2024    51
Name: state, dtype: int64


,state,year,unemployment_rate
53,AK,1980,9.5


In [5]:
# =====================================================
# Merge into final panel + save
# =====================================================

panel = (
    saipe_clean
    .merge(laus_clean, on=["state", "year"], how="left", validate="1:1")
    .sort_values(["state", "year"])
    .reset_index(drop=True)
)

out_parquet = PROCESSED / "state_year_panel.parquet"
out_csv     = PROCESSED / "state_year_panel.csv"

panel.to_parquet(out_parquet, index=False)
panel.to_csv(out_csv, index=False)

print("Saved:", out_parquet)
print("Saved:", out_csv)
print("Missing unemployment_rate share:", float(panel["unemployment_rate"].isna().mean()))

display(panel.head(10))
print("Panel years:", int(panel["year"].min()), "to", int(panel["year"].max()))
print("How many years?", int(panel["year"].nunique()))

Saved: C:\projects\python-policy-project\data\processed\state_year_panel.parquet
Saved: C:\projects\python-policy-project\data\processed\state_year_panel.csv
Missing unemployment_rate share: 0.0


,state,state_name,state_fips,year,poverty_rate,median_income,unemployment_rate
0,AK,Alaska,02,1989,10.6,33885,7.1
1,AK,Alaska,02,1993,11.2,39431,7.8
2,AK,Alaska,02,1995,10.1,42255,7.3
3,AK,Alaska,02,1996,10.6,44797,7.5
4,AK,Alaska,02,1997,11.2,43657,7.0
5,AK,Alaska,02,1998,10.8,47177,6.3
6,AK,Alaska,02,1999,8.8,49133,6.4
7,AK,Alaska,02,2000,8.5,51433,6.3
8,AK,Alaska,02,2001,8.7,52332,6.3
9,AK,Alaska,02,2002,9.3,51844,7.2


Panel years: 1989 to 2023
How many years? 31
